In [ ]:
import numpy as np
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import mean_squared_error
import geopy.distance
import datetime
import csv

def time_from_polyline(polyline):
    return (len(polyline)-1)*15


def list_from_polyline_string(polyline_str):
    final_polyline = []
    c = 0
    while c < len(polyline_str):
        if c == 0 or c == len(polyline_str)-1:
            c += 1
        else:
            coords = []
            if polyline_str[c] == "[":
                c += 1
                d = c
                while polyline_str[d] != ",":
                    d += 1
                coords.append(float(polyline_str[c:d]))
                d += 1
                c = d
                while polyline_str[d] != "]":
                    d += 1
                coords.append(float(polyline_str[c:d]))
                d += 1
                c = d

                final_polyline.append(coords)
                c += 1
            else:
                c += 1

    return final_polyline


def distance_gps_coordinates(c1, c2):
    return geopy.distance.vincenty(c1, c2).miles


def distance_from_polyline(polyline):
    final_dist = 0.00
    for i in range(len(polyline)):
        if i == len(polyline)-1:
            break
        else:
            final_dist += distance_gps_coordinates(polyline[i], polyline[i+1])

    return final_dist

# Returns hour and day of the week
def from_unix_timestamp(ts):
    dt = datetime.datetime.fromtimestamp(ts)
    hour = dt.hour
    week_day = dt.weekday()
    # print(hour, week_day)
    return hour, week_day


In [ ]:
taxi_stand_id_to_lat_lon = {}
with open('data/metaData_taxistandsID_name_GPSlocation.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            # print(row)
            line_count += 1
        else:
            taxi_stand_id_to_lat_lon[int(row[0])] = [float(row[2]), float(row[3])]
            line_count += 1

    print(f'Processed {line_count} lines.')


Processed 64 lines.


In [40]:
trip_metrics = []
target_distance_yi = []
with open('data/train.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            # print(row)
            line_count += 1
        elif line_count == 100000:
            break
        else:
            if row[7] == "True" or row[1] == "A" or row[1] == "C" or row[3] == '':
                continue
            timestamp = int(row[5])
            hour, weekday = from_unix_timestamp(timestamp)
            origin_stand_lat = taxi_stand_id_to_lat_lon[int(row[3])][0]
            origin_stand_lon = taxi_stand_id_to_lat_lon[int(row[3])][1]
            distance_yi = distance_from_polyline(list_from_polyline_string(row[8]))

            trip_metrics.append([hour+1, weekday+1, origin_stand_lat, origin_stand_lon])
            target_distance_yi.append(distance_yi)
            line_count += 1
            if line_count % 10000 == 0:
                print(line_count)
    print(trip_metrics[0:1000])
    print(f'Processed {line_count} lines.')


10000


20000


30000


40000


50000


60000


70000


80000


90000


100000
[[21, 7, 41.1599801853, -8.64198392478], [21, 7, 41.1570891314, -8.6284679801], [21, 7, 41.1632224305, -8.58404677278], [21, 7, 41.1607148883, -8.60424608207], [21, 7, 41.1549650972, -8.61321698848], [21, 7, 41.1460158298, -8.61257471887], [21, 7, 41.168317889, -8.68917996027], [22, 7, 41.1570891314, -8.6284679801], [22, 7, 41.168317889, -8.68917996027], [22, 7, 41.1570891314, -8.6284679801], [22, 7, 41.1457188058, -8.61070701502], [22, 7, 41.1460158298, -8.61257471887], [22, 7, 41.1570891314, -8.6284679801], [22, 7, 41.1405168841, -8.6159521911], [22, 7, 41.1486275073, -8.58587660305], [22, 7, 41.1405168841, -8.6159521911], [22, 7, 41.1483209068, -8.61960347647], [22, 7, 41.1460158298, -8.61257471887], [22, 7, 41.1738135714, -8.65449600832], [23, 7, 41.1442525263, -8.60571961615], [23, 7, 41.1517836424, -8.62775942991], [23, 7, 41.1457188058, -8.61070701502], [22, 7, 41.1614038495, -8.62112197229], [23, 7, 41.1460158298, -8.61257471887], [22, 7, 41.1607148883, -8.60424608207], 

In [41]:
xi = np.array(trip_metrics)
yi = np.array(target_distance_yi)
reg = linear_model.LinearRegression()
reg.fit(xi, yi)
distance_pred = reg.predict(xi)
print(distance_pred)
mse = mean_squared_error(yi, distance_pred)
print(reg.coef_)
print(type(reg.coef_))
print(reg.intercept_)
print(reg.score(xi, yi))
print(mse)

[3.39056741 3.54167211 4.04072234 ... 3.73781426 3.97034185 4.05361702]
[-0.02080835  0.02199497  0.15560052 11.21301988]
<class 'numpy.ndarray'>
94.17180130542822
0.003215616224810036
22.077301108624088


In [ ]:
target_distance_yi